Import and preprocessing

In [69]:
import pandas as pd
import numpy as np
import os

#Data Source: PORDATA
ROOT_FOLDER = "C:\\Users\\liamo\\Documents\\BIOINF\\SIB\\tp\\trab\\"
MORE_DATA_OUT_PATH = os.path.join(ROOT_FOLDER,"more_pordata.csv")
GETARIO_GENDER_PATH = os.path.join(ROOT_FOLDER,"getario_gender_sim.csv")

ENSINO_PATH =  os.path.join(ROOT_FOLDER,"estab_ensino2019.xlsx")
GANHO_MENSAL_PATH =  os.path.join(ROOT_FOLDER,"ganho_meansal_genero2018.xlsx")
MEDFARM_PATH =  os.path.join(ROOT_FOLDER,"habitantes_por_medico_farmac2019.xlsx")
HOSPEDES_PATH =  os.path.join(ROOT_FOLDER,"hospedes_alojamento2019.xlsx")
HOSPITAIS_PATH =  os.path.join(ROOT_FOLDER,"hospitais2018.xlsx")
PODER_COMPRA_PATH =  os.path.join(ROOT_FOLDER,"poder_compra_percapita2017.xlsx")
TAXA_ABSTENCAO_PATH = os.path.join(ROOT_FOLDER,"taxa_abstencao_assrep2019.xlsx")
UNI_PATH = os.path.join(ROOT_FOLDER,"estab_ensino_uni2020.xlsx")
GETARIO_FEM_PATH = os.path.join(ROOT_FOLDER,"getario_fem2011.xlsx")
GETARIO_MASC_PATH = os.path.join(ROOT_FOLDER,"getario_masc2011.xlsx")
#Data Source: PT covid19 github (also originally from PORDATA)
POPGETARIO_CONC_PATH = os.path.join(ROOT_FOLDER,"popres31dez19_porgetario.xlsx")

ensino = pd.read_excel(ENSINO_PATH,usecols=list(range(7)),nrows=353,skiprows=list(range(10)),header=0) 
uni = pd.read_excel(UNI_PATH,usecols=list(range(5)),nrows=353,skiprows=list(range(10)),header=0) 
ganhomes = pd.read_excel(GANHO_MENSAL_PATH,usecols=list(range(5)),nrows=353,skiprows=list(range(10)),header=0) 
medfarm = pd.read_excel(MEDFARM_PATH,usecols=list(range(4)),nrows=353,skiprows=list(range(10)),header=0) 
hospedes = pd.read_excel(HOSPEDES_PATH,usecols=list(range(4)),nrows=353,skiprows=list(range(10)),header=0) 
hospitais = pd.read_excel(HOSPITAIS_PATH,usecols=list(range(5)),nrows=353,skiprows=list(range(10)),header=0) 
pcompra = pd.read_excel(PODER_COMPRA_PATH,usecols=list(range(3)),nrows=353,skiprows=list(range(10)),header=0) 
abstencao = pd.read_excel(TAXA_ABSTENCAO_PATH,usecols=list(range(3)),nrows=353,skiprows=list(range(10)),header=0) 
getario = pd.read_excel(POPGETARIO_CONC_PATH,usecols=list(range(21)),nrows=353,skiprows=list(range(10)),header=0) 
gefem = pd.read_excel(GETARIO_FEM_PATH,usecols=list(range(19)),nrows=353,skiprows=list(range(10)),header=0)
gemasc = pd.read_excel(GETARIO_MASC_PATH,usecols=list(range(19)),nrows=353,skiprows=list(range(10)),header=0)

In [70]:
rep_dict = {"Ç":"C","Â":"A","Ã":"A","Á":"A","À":"A","É":"E","Ê":"E","Í":"I","Ó":"O","Ô":"O","Ú":"U"," ":"-"}
# fix header and upper case values
def fixdf(df):
    df.rename(columns = {'Unnamed: 0':'Âmbito Geográfico','Unnamed: 1':'Concelho'}, inplace = True)
    df['Concelho'] = df['Concelho'].str.upper()
    df['Concelho'].replace(rep_dict,regex=True,inplace=True)
    df = df.drop([0],axis=0)
    df = df[df["Âmbito Geográfico"]=="Município"].drop(["Âmbito Geográfico"],axis=1)
    return df
#fix age groups
def fixages(df):
    df["0_9"] = df['0-04']+df["05-09"]
    df["10_19"] = df['10-14']+df['15-19']
    df["20_29"] = df['20-24']+df['25-29']
    df["30_39"] = df['30-34']+df['35-39']
    df["40_49"] = df['40-44']+df['45-49']
    df["50_59"] = df['50-54']+df['55-59']
    df["60_69"] = df['60-64']+df['65-69']
    if "75 ou mais" not in df.columns:
        df["70_79"] = df['70-74']+df['75-79']
        df["80_plus"] = df['80-84']+df['85 ou mais']
        df = df.drop(['0-04', '05-09', '10-14', '15-19', '20-24','25-29', '30-34', '35-39', '40-44', '45-49', '50-54',
                                  '55-59', '60-64','65-69', '70-74', '75-79', '80-84', '85 ou mais'],axis=1)
    else:
        df["70_plus"] = df['70-74']+df['75 ou mais']
        df = df.drop(['0-04', '05-09', '10-14', '15-19', '20-24','25-29', '30-34', '35-39', '40-44', '45-49', '50-54',
                          '55-59', '60-64','65-69', '70-74', '75 ou mais'],axis=1)
    return df

conc_get = fixages(fixdf(getario))
gefem = fixages(fixdf(gefem))
gemasc = fixages(fixdf(gemasc))

ensino = fixdf(ensino)
uni = fixdf(uni)
ganhomes = fixdf(ganhomes)
medfarm = fixdf(medfarm)
medfarm.rename(columns = {'Unnamed: 2':'Medico','Unnamed: 3':'Farmaceutico'}, inplace = True) 
hospedes = fixdf(hospedes)
hospitais = fixdf(hospitais)
pcompra = fixdf(pcompra)
pcompra.rename(columns = {'Unnamed: 2':'PoderCompra'}, inplace = True)
abstencao = fixdf(abstencao)
abstencao.rename(columns = {'Unnamed: 2':'Abstencao'}, inplace = True)

In [56]:
#normalize by population, per 100k people???
# hospitais.iloc[:,1:] = 10*1000*hospitais.iloc[:,1:].div(conc_get["Total"],axis=0)
# ensino.iloc[:,1:] = 10*1000*ensino.iloc[:,1:].div(conc_get["Total"],axis=0) 
# uni.iloc[:,1:] = 10*1000*uni.iloc[:,1:].div(conc_get["Total"],axis=0) 
# hospedes.iloc[:,1:] = 10*1000*hospedes.iloc[:,1:].div(conc_get["Total"],axis=0)

# hospitais.iloc[:,1:] = hospitais.iloc[:,1:].round(1)
# ensino.iloc[:,1:] = ensino.iloc[:,1:].round(1)
# uni.iloc[:,1:] = uni.iloc[:,1:].round(1)
# hospedes.iloc[:,1:] = hospedes.iloc[:,1:].round(1)

In [71]:
age = pd.DataFrame()
age["2019"] = conc_get["Total"]
age["2011"] = gefem["Total"]+gemasc["Total"]
age["FIX"] = age["2019"]/age["2011"] #variation since 2011

def updateages(df):
    for col in df.columns[2:]: 
        df[col] = (df[col]*age["FIX"]).round(0)
    return df.drop(["Total"],axis=1)
         
gefem2 = updateages(gefem)
gemasc2 = updateages(gemasc)
ge_gender = pd.concat((gefem2.iloc[:,1:],gemasc2.iloc[:,1:]),axis=1,keys=["F","M"])
ge_gender.index = gefem2["Concelho"]
print(ge_gender.head())
ge_gender.to_csv(GETARIO_GENDER_PATH)

In [11]:
# print(uni.head())
# print(ensino.head())
# print(ganhomes.head())
# print(medfarm.head())
# print(hospedes.head())
# print(hospitais.head())
# print(pcompra.head())
# print(abstencao.head())
# print(conc_get.head())

Concatenate and export data

In [12]:
more_data = pd.concat((hospitais.iloc[:,1:],medfarm.iloc[:,1:],ensino.iloc[:,1:],uni.iloc[:,1:],abstencao.iloc[:,1:],ganhomes.iloc[:,1:],
                      pcompra.iloc[:,1:],hospedes.iloc[:,1:]),axis=1,keys=["Hospitais","MedFarm","Ensino","Universidades",
                                                                           "Abstencao","GanhoMes","PoderCompra","Hospedes"])
more_data.index = hospitais["Concelho"]
print(more_data.head())
more_data.to_csv(MORE_DATA_OUT_PATH)

                  Hospitais                 MedFarm               \
                      Total Oficial Privado  Medico Farmaceutico   
Concelho                                                           
ARCOS-DE-VALDEVEZ         0       0       0   476.1        837.9   
CAMINHA                   0       0       0   193.6       1058.3   
MELGACO                   0       0       0   386.6        738.0   
MONCAO                    0       0       0   279.5        851.7   
PAREDES-DE-COURA          0       0       0   657.5       1424.6   

                                Ensino                           \
                  Educação Pré-Escolar Ensino Básico - 1º Ciclo   
Concelho                                                          
ARCOS-DE-VALDEVEZ                   10                        4   
CAMINHA                             11                       10   
MELGACO                              3                        2   
MONCAO                               6               